In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/abhishek/Library/Application Support/sagemaker/config.yaml


In [2]:
!pip install --upgrade --force-reinstall scikit-learn

  Using cached scikit_learn-1.4.1.post1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.12.0-cp39-cp39-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.3.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.1.post1-cp39-cp39-macosx_10_9_x86_64.whl (11.6 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
Using cached scipy-1.12.0-cp39-cp39-macosx_10_9_x86_64.whl (38.9 MB)
Using cached threadpoolctl-3.3.0-py3-none-any.whl (17 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.3.0
    Uninstalling threadpoolctl-3.3.0:
      Successfully uninstalled threadpoolctl-3.3.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      

In [3]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import pearsonr
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
import seaborn as sns

In [4]:
import redshift_connector

In [5]:
redshift_endpoint = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'
redshift_dbname = 'factihealth'
redshift_port = 5439
redshift_user = 'fh_user'
redshift_pass = 'Facti@874'

In [6]:
conn = redshift_connector.connect(
     host=redshift_endpoint,
     database=redshift_dbname,
     port=int(redshift_port),
     user=redshift_user,
     password=redshift_pass
  )

In [7]:
def read_table(query):
   
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    
    dataframe = pd.DataFrame(rows, columns=column_names)
    
    cursor.close()

    return dataframe

In [8]:
admission_query = """
SELECT subject_id, hadm_id, admittime, dischtime, deathtime, admission_type, insurance, race
FROM mimic.admissions
"""

patient_query = """
SELECT subject_id, gender, anchor_age, dod
FROM mimic.patients
"""

diagnoses_icd_query = """
SELECT *
FROM mimic.diagnosis_icd
"""

icustays_query = """
SELECT *
FROM mimic.icustays
"""

In [9]:
admits = read_table(admission_query)

# Convert dates to a proper format
admits.admittime = pd.to_datetime(admits.admittime)
admits.dischtime = pd.to_datetime(admits.dischtime)
admits.deathtime = pd.to_datetime(admits.deathtime)

admits.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,insurance,race
0,12024697,20302177,2109-12-14 22:50:00,2110-01-15 14:53:00,NaT,EW EMER.,Other,WHITE
1,15350437,20383396,2110-01-11 08:02:00,2110-01-12 18:45:00,NaT,EU OBSERVATION,Other,WHITE
2,18106347,24305596,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,EW EMER.,Other,WHITE
3,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,NaT,EW EMER.,Other,UNABLE TO OBTAIN
4,11274131,28390257,2110-01-12 22:11:00,2110-01-14 15:31:00,NaT,EW EMER.,Other,WHITE


In [10]:
admits['died_at_the_hospital'] = admits['deathtime'].notnull().map({True:1, False:0})

In [11]:
# Compress the number of race categories
admits['race'].replace(regex=r'^ASIAN\D*', value = 'ASIAN', inplace = True)
admits['race'].replace(regex=r'^WHITE\D*', value = 'WHITE', inplace = True)
admits['race'].replace(regex=r'^HISPANIC\D*', value = 'HISPANIC/LATINO', inplace = True)
admits['race'].replace(regex=r'^BLACK\D*', value = 'BLACK/AFRICAN AMERICAN', inplace = True)
admits['race'].replace(['UNABLE TO OBTAIN', 'OTHER', 'PATIENT DECLINED TO ANSWER', 'UNKNOWN/NOT SPECIFIED'], value = 'OTHER/UNKNOWN', inplace = True)

# take into consideration just top-5 categories with biggest value count, the others will fall into OTHER category
admits['race'].loc[~admits['race'].isin(admits['race'].value_counts().nlargest(5).index.tolist())] = 'OTHER/UNKNOWN'

In [12]:
# Compress into EMERGENCY
admits['admission_type'].replace(to_replace = 'EW EMER.', value = 'EMERGENCY', inplace = True)
admits['admission_type'].replace(to_replace = 'DIRECT EMER.', value = 'EMERGENCY', inplace = True)
admits['admission_type'].replace(to_replace = 'URGENT', value = 'EMERGENCY', inplace = True)

In [13]:
# Compress into EMERGENCY
admits['admission_type'].replace(to_replace = 'EU OBSERVATION', value = 'OBSERVATION', inplace = True)
admits['admission_type'].replace(to_replace = 'OBSERVATION ADMIT', value = 'OBSERVATION', inplace = True)
admits['admission_type'].replace(to_replace = 'DIRECT OBSERVATION', value = 'OBSERVATION', inplace = True)
admits['admission_type'].replace(to_replace = 'AMBULATORY OBSERVATION', value = 'OBSERVATION', inplace = True)

In [14]:
patients = read_table(patient_query)

patients.head()

,subject_id,gender,anchor_age,dod
0,10000048,F,23,None
1,10000068,F,19,None
2,10000108,M,25,None
3,10000178,F,59,None
4,10000280,M,20,None


In [15]:
admits_patients = pd.merge(admits, patients, how = 'inner', on = 'subject_id')

In [16]:
diag_icds = read_table(diagnoses_icd_query)
diag_icds.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10158684,22072764,1,0020,9
1,10427677,23641430,1,0020,9
2,11088311,27128981,1,0020,9
3,12916672,24528914,1,0020,9
4,10284802,25193580,3,0030,9


In [17]:
diag_icds['icd_version'].unique()

array(['9', '10'], dtype=object)

In [18]:
diag_icds = diag_icds[diag_icds['icd_version'] == '9']
diag_icds.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10158684,22072764,1,0020,9
1,10427677,23641430,1,0020,9
2,11088311,27128981,1,0020,9
3,12916672,24528914,1,0020,9
4,10284802,25193580,3,0030,9


In [19]:
# Filter out E and V codes from ICD9 codes since processing will be done on the numeric first 3 values
diag_icds['recode'] = diag_icds['icd_code']
diag_icds['recode'] = diag_icds['recode'][~diag_icds['recode'].str.contains("[a-zA-Z]").fillna(False)]
diag_icds['recode'].fillna(value='999', inplace=True)

# Take in consideration just the first 3 integers of the ICD9 code
diag_icds['recode'] = diag_icds['recode'].str.slice(start=0, stop=3, step=1)
diag_icds['recode'] = diag_icds['recode'].astype(int)

In [20]:
# ICD-9 Main Category ranges
icd9_ranges = [(1, 140), (140, 240), (240, 280), (280, 290), (290, 320), (320,390),
               (390, 460), (460, 520), (520, 580), (580, 630), (630, 680),(680, 710),
               (710, 740), (740, 760), (760, 780), (780, 800), (800, 1000),(1000, 2000)]

# Associated category names
diag_dict = {0: 'infectious', 1: 'neoplasms', 2: 'endocrine', 3: 'blood',
             4: 'mental', 5: 'nervous', 6: 'circulatory', 7: 'respiratory',
             8: 'digestive', 9: 'genitourinary', 10: 'pregnancy', 11: 'skin',
             12: 'muscular', 13: 'congenital', 14: 'prenatal', 15: 'misc',
             16: 'injury', 17: 'misc'}

# Re-code in terms of integer
for num, cat_range in enumerate(icd9_ranges):
    diag_icds['recode'] = np.where(diag_icds['recode'].between(cat_range[0],cat_range[1]), num, diag_icds['recode'])
    
# Convert integer to category name using diag_dict
diag_icds['super_category'] = diag_icds['recode'].replace(diag_dict)

In [21]:
hadm_list = diag_icds.groupby('hadm_id')['super_category'].apply(list).reset_index()

In [22]:
hadm_item = pd.get_dummies(hadm_list['super_category'].explode()).groupby(hadm_list['hadm_id']).sum()
hadm_item.reset_index(inplace=True)

In [23]:
hadm_item.shape

(276803, 18)

In [24]:
admits_patients_diag = pd.merge(admits_patients, hadm_item, how='inner', on='hadm_id')

In [25]:
#convert dates
admits_patients_diag.admittime = pd.to_datetime(admits_patients_diag.admittime)
admits_patients_diag.dischtime = pd.to_datetime(admits_patients_diag.dischtime)
admits_patients_diag.deathtime = pd.to_datetime(admits_patients_diag.deathtime)

In [26]:
# Create LOS attribute converting timedelta type into float 'days', 86400 seconds in a day
admits_patients_diag['los'] = (admits_patients_diag['dischtime'] - admits_patients_diag['admittime']).dt.total_seconds()/86400

In [27]:
admits_patients_diag = admits_patients_diag[admits_patients_diag['los'] >= 0]

In [28]:
admits_patients_diag.shape

(553412, 30)

In [29]:
# Hospital LOS metrics for later comparison
actual_mean_los = admits_patients_diag['los'].loc[admits_patients_diag['died_at_the_hospital'] == 0].mean()
actual_median_los = admits_patients_diag['los'].loc[admits_patients_diag['died_at_the_hospital'] == 0].median()

print(actual_mean_los)
print(actual_median_los)

4.0477820533043944
2.540277777777778


In [30]:
age_ranges = [(0, 13), (14, 36), (37, 56), (57, 100)]
for num, cat_range in enumerate(age_ranges):
    admits_patients_diag['anchor_age'] = np.where(admits_patients_diag['anchor_age'].between(cat_range[0],cat_range[1]),num, admits_patients_diag['anchor_age'])

age_dict = {0: 'NEWBORN', 1: 'YOUNG_ADULT', 2: 'MIDDLE_ADULT', 3: 'SENIOR'}
admits_patients_diag['anchor_age'] = admits_patients_diag['anchor_age'].replace(age_dict)

In [31]:
icustays = read_table(icustays_query)
icustays.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,18106347,24305596,30588857,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-11 10:16:06,2110-01-12 17:17:47,1.292836
1,17195991,23542772,38319097,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-01-12 00:54:00,2110-01-14 22:07:39,2.884479
2,18780420,28977824,32140208,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2110-01-17 00:28:00,2110-01-18 01:21:22,1.037060
3,15092543,28768907,38339174,Neuro Intermediate,Neuro Intermediate,2110-01-17 22:49:13,2110-01-18 17:18:51,0.770579
4,13201095,28453791,39953418,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2110-01-18 14:46:27,2110-01-25 12:42:11,6.913704


In [32]:
icustays['first_careunit'].replace({'Cardiac Vascular Intensive Care Unit (CVICU)': 'Other-ICU', 'Coronary Care Unit (CCU)': 'Other-ICU', 
                                    'Medical Intensive Care Unit (MICU)': 'Other-ICU','Medical/Surgical Intensive Care Unit (MICU/SICU)': 'Other-ICU',
                                    'Neuro Stepdown': 'Other-ICU', 'Surgical Intensive Care Unit (SICU)': 'Other-ICU', 
                                    'Trauma SICU (TSICU)': 'Other-ICU'}, inplace=True)
icustays['category'] = icustays['first_careunit']
icu_list = icustays.groupby('hadm_id')['category'].apply(list).reset_index()

In [33]:
icu_item = pd.get_dummies(icu_list['category'].explode()).groupby(icu_list['hadm_id']).sum()
icu_item[icu_item >= 1] = 1
icu_item.reset_index(inplace=True)

In [34]:
# Merge ICU data with main dataFrame
final_df = admits_patients_diag.merge(icu_item, how='outer', on='hadm_id')

In [35]:
# Replace NaNs with 0
final_df['Neuro Intermediate'].fillna(value=0, inplace=True)
final_df['Neuro Surgical Intensive Care Unit (Neuro SICU)'].fillna(value=0, inplace=True)
final_df['Other-ICU'].fillna(value=0, inplace=True)

In [36]:
# Remove deceased persons as they will skew LOS result
final_df = final_df[final_df['died_at_the_hospital'] == 0.0]

In [42]:
final_df.shape

(543078, 33)

In [44]:
final_df['hadm_id'].drop_duplicates(keep='first',inplace=True)
print(final_df.shape)

(543078, 33)


In [47]:
final_df[final_df['hadm_id']==20000019]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,insurance,race,died_at_the_hospital,gender,...,neoplasms,nervous,pregnancy,prenatal,respiratory,skin,los,Neuro Intermediate,Neuro Surgical Intensive Care Unit (Neuro SICU),Other-ICU
312544,10467237.0,20000019,2159-03-20 21:08:00,2159-03-23 16:54:00,NaT,EMERGENCY,Other,HISPANIC/LATINO,0.0,F,...,0.0,0.0,0.0,0.0,1.0,0.0,2.823611,0.0,0.0,0.0
312545,10467237.0,20000019,2159-03-20 21:08:00,2159-03-23 16:54:00,NaT,EMERGENCY,Other,HISPANIC/LATINO,0.0,F,...,0.0,0.0,0.0,0.0,1.0,0.0,2.823611,0.0,0.0,0.0


In [37]:
model_df = final_df.drop(columns=['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'died_at_the_hospital', 'dod'])

In [38]:
# Convert gender into numeric boolean attribute
model_df['gender'].replace({'M': 0, 'F':1}, inplace=True)

In [39]:
# Create dummy columns for categorical variables
prefix_cols = ['ADM', 'INS', 'RAC', 'AGE']
dummy_cols = ['admission_type', 'insurance','race', 'anchor_age']
model_df = pd.get_dummies(model_df, prefix=prefix_cols, columns=dummy_cols)

In [40]:
# Target Variable (Length-of-Stay)
LOS = model_df['los'].values
# Prediction Features
features = model_df.drop(columns=['los'])

In [41]:
# Split into train 80% and test 20%
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    LOS, 
                                                    test_size = .20, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 434462 samples.
Testing set has 108616 samples.
